In [1]:
!pip install git+https://github.com/emres/turkish-deasciifier.git

  Cloning https://github.com/emres/turkish-deasciifier.git to /private/var/folders/pz/6fz4ncm13qv_hnd5gjbql7xr0000gn/T/pip-req-build-nzif3y_c
  Running command git clone -q https://github.com/emres/turkish-deasciifier.git /private/var/folders/pz/6fz4ncm13qv_hnd5gjbql7xr0000gn/T/pip-req-build-nzif3y_c
  Created wheel for Turkish-Deasciifier: filename=Turkish_Deasciifier-0.1-py3-none-any.whl size=90359 sha256=ebba1e16c0e669b7e7d7269ae3f9081368bc42b62c122dc8290b97b18eedf871
  Stored in directory: /private/var/folders/pz/6fz4ncm13qv_hnd5gjbql7xr0000gn/T/pip-ephem-wheel-cache-bcrbwyzj/wheels/84/22/9a/d59c9c6313c39cf371642fdf5e76b9de083631aa716c14c2a7
Successfully built Turkish-Deasciifier


In [4]:
import re
from turkish.deasciifier import Deasciifier

### Format sorunları
Unicode, dijital ve geleneksel medyada her bir karakter ve sembolleri benzersiz bir rakam yardımıyla oluşturmak için geliştirilen bir metin standartıdır. Unicode’un geliştirilmesinin asıl amacı ASCII (American Standart Code for Information Interchange) karakter kodlamasının daha gelişmiş ve stratejik bir sürümünün oluşturulabilmesidir.

- ASCII yalnızca Latin alfabesi için kullanılabilir, unicode evrenseldir
- Python 'string' verisi unicode formatındadır.

*çiçek*   ->  *Ã§iÃ§ek*

**Deasciifier**
- ASCII formatındaki metinleri unicode formatına çevirmek için
- İngilizce klavyeyle yazılmış metinleri düzeltmek / hatalı karakterleri Türkçe karakterlerle değiştirmek için

*Dogal Dil Isleme* -> *Doğal Dil İşleme* 

In [32]:
def deasciify(text):
    deasciifier = Deasciifier(text)
    text = deasciifier.convert_to_turkish()
    return text

In [33]:
text = 'Dogal Dil Isleme'
deasciify(text)

'Doğal Dil İşleme'

### Metin Temizleme
İşlenecek metinler çoğunlukla 'pis' veridir.
- Sistematik hatalar (cümle başı ve sonlarındaki gereksiz boşluklar, vs.)
- Yazım hataları (gramer hataları, gereksiz boşluk kullanımı, vs.) -> ('Oluur')
- Birden fazla ünlem işareti
- Büyük/küçük harflerin yanlış kullanımı -> 'OKul'
- Rakam kullanımı

Başka problemler:
- Emoji kullanımı
- Yabancı sözcük kullanımı

Emoji ve yabancı sözcük kullanımı duruma bağlı olarak tolere edilebilir, hatta yararlı olabilir.

*Ne biliyim bende olur dedim :/:/* -> *Ne bileyim ben de olur dedim* 

In [65]:
# Örnek cümle
text = '''Ben;de    dedim  ki
olabilir1.
'''
text

'Ben;de    dedim  ki\nolabilir1.\n'

In [66]:
# Cümlenin başındaki ve sonundaki gereksiz boşlukları çıkar
text = text.strip()
text

'Ben;de    dedim  ki\nolabilir1.'

In [67]:
# Bütün karakterleri küçük harfe dönüştür
text = text.lower()
text

'ben;de    dedim  ki\nolabilir1.'

In [68]:
# Rakamları sil
text = re.sub(r'\d+', '', text)
text

'ben;de    dedim  ki\nolabilir.'

In [69]:
# Noktalama işaretleriyle harfler arasına boşluk ekle
text = re.sub( r'([a-zA-Z])([:;,.!?])', r'\1 \2', text)
text = re.sub( r'([:;,.!?])([a-zA-Z])', r'\1 \2', text)
text

'ben ; de    dedim  ki\nolabilir .'

In [70]:
# Gereksiz tümboşlukları sil
text = re.sub(r"\s+", ' ', text)
text

'ben ; de dedim ki olabilir .'

### Metin Temizleme
Yukarıdaki işlemleri tek bir fonksiyon altında toplayabiliriz.

In [71]:
def clean_text(text):
    text = text.strip()
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    
    text = re.sub( r'([a-zA-Z])([:;,.!?])', r'\1 \2', text)
    text = re.sub( r'([:;,.!?])([a-zA-Z])', r'\1 \2', text)
    
    text = re.sub(r"\s+", ' ', text)
    return text

### Metin Ön-İşleme
Dil modellerini eğitirken performansı arttırmak ve metinleri standart hale getirmek için uygulayacabileceğimiz başka adımlar da var:

- Noktalama işaretlerinin tamamının veya bir kısmının kaldırılması
- Çok sık kullanılan kelimelerin (stopwords) kaldırılması
- Kısaltma kelimeleri (ünvanlar, vs.) tam haline getirmek
- Kelimeleri kök haline getirmek ('kitabı', 'kitaplarım' -> 'kitap')

Hangi ön-işleme adımlarını kullanacağımız eğittimiz dil modelinin özelliklerine ve kullanım amacımıza bağlı. Örneğin Word2Vec gibi bir kelime temsil modelinde noktalama işaretleri ve rakamları tutmamızın hem bir amacı olmaz hem de model performansını düşürür.

In [2]:
def preprocess_text(text):

  return w